In [1]:
import os
import requests
import re
from selenium import webdriver
import json
from datetime import date
import pandas as pd

In [2]:
# Get the actual date
today = date.today()

In [3]:
# Use Chrome Browser
options = webdriver.ChromeOptions()

# Open a chrome without GI
options.add_argument('headless')

# Put a high resolution so we can reach more content on the page
options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(chrome_options=options)

# Open the page
midias_link = "https://www.sc.gov.br/noticias/temas/coronavirus"
driver.get(midias_link)

/home/alonkin/anaconda3/envs/development/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [4]:
# Get all articles on this page
element_rows = driver.find_elements_by_class_name('items-row')

In [5]:
# Creates a dictonary to transform month (text) to month (number)
month_translator = {'Janeiro': 1,'Fevereiro': 2,'Março': 3,'Abril': 4,'Maio': 5,'Junho': 6,'Julho': 7, 'Agosto': 8,'Setembro': 9, 'Outubro': 10, 'Novembro': 11, 'Dezembro': 12}

In [6]:
last_articles_about_corona = []

# Search for every article with the term "confirm" 
# and then save the links into a list
for rows in element_rows:
    link = rows.find_element_by_tag_name("a")
    date = rows.find_element_by_class_name('xtt-news-list-item-date')
    
    contains = 'confirm'.lower() in rows.find_element_by_tag_name("a").text.lower()
    date_split = date.text.split(' ')
    day = date_split[0]
    month = month_translator[date_split[1]]
    year = date_split[2]
    
    # Search only for the lastest article
    latest = (str(today.day) == day) & (today.month == month) & (str(today.year) == year)
    latest = True
    if(contains & latest):
        last_articles_about_corona.append(link.get_attribute('href'))

In [8]:
# Get the first one on the how
link_one = last_articles_about_corona[0]
driver.get(link_one)

In [9]:
# Load a file with the name of all Santa Catarina's cities
with open('estados-cidades.json') as f:
  data = json.load(f)

In [10]:
sc_cities = []
# Create an array with all cities
for estados in data['estados']:
    if estados['sigla'] == 'SC':
        sc_cities = estados['cidades']

In [11]:
body = driver.find_element_by_xpath('//div[@itemprop="articleBody"]')

In [18]:
# Creates a dictionary that will be used to create the DataFrame
number_of_confirmed_cases_by_city = {'DataLeitura': today.strftime("%d/%m/%Y")}

# Search for the name of all cities and add the amount of cases into the
# Dictionary. If the name is not listed, then we consider it 0 
for city in sc_cities:
    city_name_on_text_index = body.text.find(city)
    have_confirmed_cases = True if city_name_on_text_index > 0 else False  
    number_of_cases = 0
    if(have_confirmed_cases):
        end_character_index = body.text.find(')', city_name_on_text_index)
        city_n_cases = body.text[city_name_on_text_index:end_character_index + 1]
        number_of_cases =  city_n_cases[city_n_cases.find('(')+1 : city_n_cases.find(')')]
    number_of_confirmed_cases_by_city[city] = number_of_cases

In [22]:
pd.DataFrame([confirmed_by_city])

,DataLeitura,Abdon Batista,Abelardo Luz,Agrolândia,Agronômica,Água Doce,Águas de Chapecó,Águas Frias,Águas Mornas,Alfredo Wagner,...,Vargem,Vargem Bonita,Vidal Ramos,Videira,Vitor Meireles,Witmarsum,Xanxerê,Xavantina,Xaxim,Zortéa
0,23/03/2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
